# 🎯 RAFT Dataset Creation

This notebook creates the final RAFT (Retrieval Augmented Fine-Tuning) dataset.

**RAFT Format:**
- Each example has 1 oracle document (contains answer) + 4 distractor documents
- 80% examples: oracle + distractors
- 20% examples: only distractors (teach model to say "I don't know")

**Input:** `qa_pairs.jsonl`, `chunks.jsonl`
**Output:** `train.jsonl`, `val.jsonl`

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q sentence-transformers faiss-cpu tqdm

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paths
DATA_DIR = "/content/drive/MyDrive/RAFT_dental_data"
CHUNKS_FILE = f"{DATA_DIR}/chunks.jsonl"
QA_FILE = f"{DATA_DIR}/qa_pairs.jsonl"
OUTPUT_DIR = f"{DATA_DIR}/raft_dataset"

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Verify inputs
for f in [CHUNKS_FILE, QA_FILE]:
    if os.path.exists(f):
        print(f"✓ Found {os.path.basename(f)}")
    else:
        print(f"✗ Missing {os.path.basename(f)}")

## 2. Load Data

In [ ]:
import json
import random
from tqdm.notebook import tqdm

# Load chunks
chunks = []
chunk_by_id = {}
with open(CHUNKS_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        chunk = json.loads(line)
        chunks.append(chunk)
        chunk_by_id[chunk['chunk_id']] = chunk

print(f"Loaded {len(chunks)} chunks")

# Load Q&A pairs
qa_pairs = []
with open(QA_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        qa_pairs.append(json.loads(line))

print(f"Loaded {len(qa_pairs)} Q&A pairs")

## 3. Create Embedding Index for Distractor Selection

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load embedding model (medical domain)
print("Loading embedding model...")
embed_model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
print(f"✓ Model loaded: {embed_model.get_sentence_embedding_dimension()}D embeddings")

In [ ]:
# Generate embeddings for all chunks
print("Generating chunk embeddings...")

chunk_texts = [c['text'] for c in chunks]
chunk_embeddings = embed_model.encode(
    chunk_texts,
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
)

print(f"Embeddings shape: {chunk_embeddings.shape}")

In [ ]:
# Build FAISS index
print("Building FAISS index...")

dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner product (cosine after normalization)

# Normalize embeddings for cosine similarity
faiss.normalize_L2(chunk_embeddings)
index.add(chunk_embeddings)

print(f"✓ FAISS index built with {index.ntotal} vectors")

## 4. RAFT Dataset Builder

In [ ]:
from typing import List, Dict, Optional
from dataclasses import dataclass

@dataclass
class RAFTConfig:
    """Configuration for RAFT dataset creation."""
    num_distractors: int = 4          # Number of distractor documents
    oracle_ratio: float = 0.8         # Ratio of examples with oracle
    max_context_length: int = 3000    # Max tokens for context
    train_ratio: float = 0.9          # Train/val split


class RAFTDatasetBuilder:
    """Build RAFT dataset from Q&A pairs and chunks."""
    
    def __init__(
        self,
        chunks: List[Dict],
        chunk_embeddings: np.ndarray,
        faiss_index: faiss.Index,
        embed_model: SentenceTransformer,
        config: RAFTConfig = None
    ):
        self.chunks = chunks
        self.chunk_embeddings = chunk_embeddings
        self.index = faiss_index
        self.embed_model = embed_model
        self.config = config or RAFTConfig()
        
        # Create chunk lookup
        self.chunk_by_id = {c['chunk_id']: i for i, c in enumerate(chunks)}
    
    def find_distractors(
        self,
        question: str,
        oracle_idx: int,
        k: int = 10
    ) -> List[int]:
        """Find distractor documents similar to question but not the oracle."""
        # Embed question
        q_embedding = self.embed_model.encode([question], convert_to_numpy=True)
        faiss.normalize_L2(q_embedding)
        
        # Search
        _, indices = self.index.search(q_embedding, k + 5)
        
        # Filter out oracle and return top k
        distractors = [idx for idx in indices[0] if idx != oracle_idx]
        return distractors[:k]
    
    def format_context_document(self, chunk: Dict, is_oracle: bool = False) -> Dict:
        """Format a chunk as a context document."""
        return {
            "content": chunk['text'],
            "source": chunk.get('source', 'Unknown'),
            "page_number": chunk.get('page_number', 0),
            "category": chunk.get('category', 'Unknown'),
            "is_oracle": is_oracle
        }
    
    def create_raft_example(
        self,
        qa: Dict,
        include_oracle: bool = True
    ) -> Optional[Dict]:
        """Create a single RAFT example."""
        question = qa.get('question', '')
        answer = qa.get('answer', '')
        chunk_id = qa.get('chunk_id', '')
        
        if not question or not answer:
            return None
        
        # Get oracle chunk index
        oracle_idx = self.chunk_by_id.get(chunk_id, -1)
        if oracle_idx == -1:
            # Try to find by text match
            context_text = qa.get('context_text', '')
            for i, c in enumerate(self.chunks):
                if c['text'] == context_text:
                    oracle_idx = i
                    break
        
        if oracle_idx == -1:
            return None
        
        # Get distractors
        distractor_indices = self.find_distractors(
            question,
            oracle_idx,
            k=self.config.num_distractors
        )
        
        # Build context
        context = []
        
        if include_oracle:
            # Add oracle document
            oracle_chunk = self.chunks[oracle_idx]
            context.append(self.format_context_document(oracle_chunk, is_oracle=True))
            
            # Add distractors
            for idx in distractor_indices[:self.config.num_distractors]:
                context.append(self.format_context_document(self.chunks[idx], is_oracle=False))
        else:
            # Only distractors (no oracle)
            for idx in distractor_indices[:self.config.num_distractors + 1]:
                context.append(self.format_context_document(self.chunks[idx], is_oracle=False))
            
            # Modify answer for no-oracle case
            answer = self._create_no_oracle_answer(question)
        
        # Shuffle context so oracle isn't always first
        random.shuffle(context)
        
        return {
            "question": question,
            "context": context,
            "answer": answer,
            "has_oracle": include_oracle,
            "category": qa.get('category', 'Unknown'),
            "difficulty": qa.get('difficulty', 'medium'),
            "question_type": qa.get('type', 'unknown')
        }
    
    def _create_no_oracle_answer(self, question: str) -> str:
        """Create answer for when oracle is not present."""
        return (
            "Based on the provided documents, I cannot find sufficient information "
            "to accurately answer this question. The documents do not contain "
            "relevant content about this specific topic. Please provide additional "
            "context or consult a more specialized dental reference."
        )
    
    def build_dataset(
        self,
        qa_pairs: List[Dict],
        output_dir: str
    ) -> Dict[str, int]:
        """Build complete RAFT dataset."""
        print(f"Building RAFT dataset from {len(qa_pairs)} Q&A pairs...")
        
        raft_examples = []
        failed = 0
        
        for qa in tqdm(qa_pairs, desc="Creating examples"):
            # Decide if this example gets oracle
            include_oracle = random.random() < self.config.oracle_ratio
            
            example = self.create_raft_example(qa, include_oracle)
            
            if example:
                raft_examples.append(example)
            else:
                failed += 1
        
        print(f"Created {len(raft_examples)} examples ({failed} failed)")
        
        # Split train/val
        random.shuffle(raft_examples)
        split_idx = int(len(raft_examples) * self.config.train_ratio)
        
        train_examples = raft_examples[:split_idx]
        val_examples = raft_examples[split_idx:]
        
        # Save
        train_path = os.path.join(output_dir, "train.jsonl")
        val_path = os.path.join(output_dir, "val.jsonl")
        
        with open(train_path, 'w', encoding='utf-8') as f:
            for ex in train_examples:
                f.write(json.dumps(ex, ensure_ascii=False) + "\n")
        
        with open(val_path, 'w', encoding='utf-8') as f:
            for ex in val_examples:
                f.write(json.dumps(ex, ensure_ascii=False) + "\n")
        
        print(f"\n✓ Saved {len(train_examples)} train examples to {train_path}")
        print(f"✓ Saved {len(val_examples)} val examples to {val_path}")
        
        return {
            "total": len(raft_examples),
            "train": len(train_examples),
            "val": len(val_examples),
            "failed": failed,
            "with_oracle": sum(1 for e in raft_examples if e['has_oracle']),
            "without_oracle": sum(1 for e in raft_examples if not e['has_oracle'])
        }

## 5. Build RAFT Dataset

In [ ]:
# Configure RAFT
config = RAFTConfig(
    num_distractors=4,      # 4 distractor docs per example
    oracle_ratio=0.8,       # 80% have oracle, 20% only distractors
    train_ratio=0.9         # 90% train, 10% val
)

print("RAFT Configuration:")
print(f"  Distractors per example: {config.num_distractors}")
print(f"  Oracle ratio: {config.oracle_ratio}")
print(f"  Train/Val split: {config.train_ratio}/{1-config.train_ratio}")

In [ ]:
# Build dataset
builder = RAFTDatasetBuilder(
    chunks=chunks,
    chunk_embeddings=chunk_embeddings,
    faiss_index=index,
    embed_model=embed_model,
    config=config
)

stats = builder.build_dataset(qa_pairs, OUTPUT_DIR)

## 6. Validate Dataset

In [ ]:
# Load and inspect train set
train_path = os.path.join(OUTPUT_DIR, "train.jsonl")
train_examples = []
with open(train_path, 'r', encoding='utf-8') as f:
    for line in f:
        train_examples.append(json.loads(line))

print(f"Train examples: {len(train_examples)}")

# Sample example
example = train_examples[0]
print("\nSample RAFT example:")
print(f"Question: {example['question']}")
print(f"Has Oracle: {example['has_oracle']}")
print(f"Context documents: {len(example['context'])}")
print(f"Answer preview: {example['answer'][:200]}...")

In [ ]:
# Show context structure
print("\nContext documents:")
for i, doc in enumerate(example['context']):
    print(f"\nDoc {i+1}:")
    print(f"  Source: {doc['source']}")
    print(f"  Page: {doc['page_number']}")
    print(f"  Is Oracle: {doc['is_oracle']}")
    print(f"  Content: {doc['content'][:100]}...")

In [ ]:
# Validate no-oracle examples
no_oracle = [e for e in train_examples if not e['has_oracle']]
print(f"\nNo-oracle examples: {len(no_oracle)}")

if no_oracle:
    sample = no_oracle[0]
    print(f"\nSample no-oracle example:")
    print(f"Question: {sample['question']}")
    print(f"Answer: {sample['answer']}")
    print(f"All is_oracle values: {[d['is_oracle'] for d in sample['context']]}")

## 7. Summary

In [ ]:
train_size = os.path.getsize(train_path) / (1024**2)
val_path = os.path.join(OUTPUT_DIR, "val.jsonl")
val_size = os.path.getsize(val_path) / (1024**2)

print("="*60)
print("RAFT DATASET CREATION COMPLETE")
print("="*60)
print(f"📊 Total examples: {stats['total']}")
print(f"📈 Train examples: {stats['train']} ({train_size:.1f} MB)")
print(f"📉 Val examples: {stats['val']} ({val_size:.1f} MB)")
print(f"✅ With oracle: {stats['with_oracle']} ({100*stats['with_oracle']/stats['total']:.1f}%)")
print(f"❌ Without oracle: {stats['without_oracle']} ({100*stats['without_oracle']/stats['total']:.1f}%)")
print(f"⚠️ Failed: {stats['failed']}")
print(f"📁 Output: {OUTPUT_DIR}")
print("="*60)
print("Next: Run 04_embedding_creation.ipynb (for FAISS index)")
print("Then: Run 05_model_training.ipynb (for QLoRA fine-tuning)")
print("="*60)